In [ ]:
pip install selenium beautifulsoup4 pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 22.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time

base_url = "https://www.shl.com"

# Parse the HTML table - replace with your actual HTML table
html = '''
<table>
                                <tbody><tr>
                                    <th class="custom__table-heading__title">Individual Test Solutions</th>
                                    <th class="custom__table-heading__general">Remote Testing</th>
                                    <th class="custom__table-heading__general">Adaptive/IRT</th>
                                    <th class="custom__table-heading__general">Test Type</th>
                                </tr>


                                        <tr data-entity-id="17">
                                            <td class="custom__table-heading__title">
                                                <a href="/solutions/products/product-catalog/view/written-english-v1/">Written English v1</a>
                                            </td>
                                            <td class="custom__table-heading__general">

                                                    <span class="catalogue__circle -yes"></span>

                                            </td>
                                            <td class="custom__table-heading__general">

                                                    <span class="catalogue__circle -yes"></span>

                                            </td>
                                            <td class="custom__table-heading__general product-catalogue__keys">

                                                    <span class="product-catalogue__key" data-has-tooltip="true" data-tooltip="productCatalogueTooltip">K</span>

                                            </td>
                                        </tr>

                                        <tr data-entity-id="18">
                                            <td class="custom__table-heading__title">
                                                <a href="/solutions/products/product-catalog/view/written-spanish/">Written Spanish</a>
                                            </td>
                                            <td class="custom__table-heading__general">

                                                    <span class="catalogue__circle -yes"></span>

                                            </td>
                                            <td class="custom__table-heading__general">

                                                    <span class="catalogue__circle -yes"></span>

                                            </td>
                                            <td class="custom__table-heading__general product-catalogue__keys">

                                                    <span class="product-catalogue__key" data-has-tooltip="true" data-tooltip="productCatalogueTooltip">K</span>

                                            </td>
                                        </tr>

                                        <tr data-entity-id="4166">
                                            <td class="custom__table-heading__title">
                                                <a href="/solutions/products/product-catalog/view/zabbix-new/">Zabbix (New)</a>
                                            </td>
                                            <td class="custom__table-heading__general">

                                                    <span class="catalogue__circle -yes"></span>

                                            </td>
                                            <td class="custom__table-heading__general">

                                            </td>
                                            <td class="custom__table-heading__general product-catalogue__keys">

                                                    <span class="product-catalogue__key" data-has-tooltip="true" data-tooltip="productCatalogueTooltip">K</span>

                                            </td>
                                        </tr>

                                        <tr data-entity-id="4297">
                                            <td class="custom__table-heading__title">
                                                <a href="/solutions/products/product-catalog/view/360-digital-report/">360 Digital Report</a>
                                            </td>
                                            <td class="custom__table-heading__general">

                                                    <span class="catalogue__circle -yes"></span>

                                            </td>
                                            <td class="custom__table-heading__general">

                                            </td>
                                            <td class="custom__table-heading__general product-catalogue__keys">

                                                    <span class="product-catalogue__key" data-has-tooltip="true" data-tooltip="productCatalogueTooltip">D</span>

                                            </td>
                                        </tr>

                                        <tr data-entity-id="744">
                                            <td class="custom__table-heading__title">
                                                <a href="/solutions/products/product-catalog/view/360-multi-rater-feedback-system-mfs/">360° Multi-Rater Feedback System (MFS)</a>
                                            </td>
                                            <td class="custom__table-heading__general">

                                                    <span class="catalogue__circle -yes"></span>

                                            </td>
                                            <td class="custom__table-heading__general">

                                            </td>
                                            <td class="custom__table-heading__general product-catalogue__keys">

                                                    <span class="product-catalogue__key" data-has-tooltip="true" data-tooltip="productCatalogueTooltip">D</span>

                                                    <span class="product-catalogue__key" data-has-tooltip="true" data-tooltip="productCatalogueTooltip">P</span>

                                            </td>
                                        </tr>


                            </tbody></table>

'''

soup = BeautifulSoup(html, 'html.parser')
test_types = {
    'A': 'Ability & Aptitude', 'B': 'Biodata & Situational Judgement',
    'C': 'Competencies', 'D': 'Development & 360',
    'E': 'Assessment Exercises', 'K': 'Knowledge & Skills',
    'P': 'Personality & Behavior', 'S': 'Simulations'
}

def extract_section(page_soup, header_text):
    """Extracts content from h4 headers based on the actual page structure"""
    header = page_soup.find('h4', string=lambda t: t and header_text.lower() in t.lower())
    if not header:
        return ''

    # Find the next p tag after this h4
    next_p = header.find_next('p')
    if not next_p:
        return ''

    return next_p.get_text(strip=True)

assessments = []

# Process the main catalog table
table = soup.find('table')
rows = table.find_all('tr')[1:]  # Skip header row

for i, row in enumerate(rows):
    cols = row.find_all('td')
    if len(cols) != 4:
        continue

    # Extract basic info from the table
    name_link = cols[0].find('a')
    assessment_name = name_link.text.strip()
    relative_url = name_link['href']
    full_url = base_url + relative_url

    remote_testing = 'Yes' if cols[1].find('span', class_='-yes') else 'No'
    adaptive_irt = 'Yes' if cols[2].find('span', class_='-yes') else 'No'

    # Process test types
    test_codes = [span.text for span in cols[3].find_all('span')]
    test_type = ', '.join([test_types.get(code, code) for code in test_codes])

    print(f"Scraping {i+1}/{len(rows)}: {assessment_name}")

    # Scrape individual assessment page
    try:
        response = requests.get(full_url)
        time.sleep(1)  # Be nice to the server
        page_soup = BeautifulSoup(response.content, 'html.parser')

        # Extract data using the correct HTML structure
        description = extract_section(page_soup, "Description")
        job_levels = extract_section(page_soup, "Job levels")

        # Handle assessment length specifically
        length_text = extract_section(page_soup, "Assessment length")
        if "=" in length_text:
            assessment_length = length_text.split('=')[-1].strip()
        else:
            assessment_length = length_text

    except Exception as e:
        print(f"Error scraping {full_url}: {str(e)}")
        description = job_levels = assessment_length = ''

    assessments.append({
        'Assessment Name': assessment_name,
        'Description': description,
        'Job Levels': job_levels,
        'Assessment Length': assessment_length,
        'Test Type': test_type,
        'Remote Testing': remote_testing,
        'Adaptive / IRT': adaptive_irt,
        'URL': full_url
    })

# Create DataFrame and export to CSV
df = pd.DataFrame(assessments)
df.to_csv('shl_assessments32.csv', index=False)
print(f"Successfully scraped {len(assessments)} assessments and saved to shl_assessments.csv")


Scraping 1/5: Written English v1
Scraping 2/5: Written Spanish
Scraping 3/5: Zabbix (New)
Scraping 4/5: 360 Digital Report
Scraping 5/5: 360° Multi-Rater Feedback System (MFS)
Successfully scraped 5 assessments and saved to shl_assessments.csv
